# explanation #

## necessity ##
confirm that you can run a fortran compiler, e.g., "gfortran",  and "f2py". 
"f2py" is a part of scipy.

## make libraries ##

change a fortran compiler if necessary and 
execute
```
$ make
```
and confirm that 
```
  myext.so
  mylib.so
  mymod.so
```
are in the directory. Probably you see so many messages and may think that it failed. If you find +x permission, 
```
-rwxrwxr-x 1 kino kino 146800  3月 16 16:37 myext.so
-rwxrwxr-x 1 kino kino 118688  3月 16 16:37 mylib.so
-rwxrwxr-x 1 kino kino 120808  3月 16 16:37 mymod.so
```
it finished successfully. 

## jupyter notebook ##
Then you can run jupyter notebook

## errors in jupyter ##
If an error occurs in running ipynb, then shutdown the process and restart it. 


# Notes #
## You can't use fortran derived types ## 
f2py **does not** natively (as of Jan 2013) directly support Fortran derived types. However, you can trick it to treat them as opaque blobs, as shown here, if you tell it how many bytes it should reserve. The exact number of bytes required may vary with compiler/compiler options, so it should be chosen big enough.

ref.
http://central.scipy.org/item/60/1/f2py-with-opaque-fortran-derived-types

## Other comments ##
- f2py can compile many many files. 
- You can't write derived types anywhere in the fortran code even if it isn't called directly from the interface code. 
- You can use module in the fortan side to save/communicate among fortran routines.
- f2py is designed so that the fortran is only a sub part of the python, which accelarates a function

# How to call fortran subroutine from python #
## fortran side ##
Explicitly add 'intent(in)' and 'intent(out)' in the definition of fortran subroutine like 
```
subroutine vavbmc(n,m,a,b,c)
  implicit none
  integer,intent(in):: n,m
  real(8),intent(in):: a(n)
  real(8),intent(in):: b(m)
  real(8),intent(out):: c(n,m)
  integer:: i,j
  do i=1,m
     do j=1,n
        c(j,i) = a(j)*b(i)
     enddo
  enddo
end subroutine vavbmc
```
f2py will make a code which allocates c(n,m) automatically in the python side.  c(:,:) must not be allocatable. Then, save it to mylib.f90
## python side ##
Then you can call this subroutine as 
```
import mylib
# a = np.array(...)
# b = np.array(...)
c = mylib.vavbmc(a,b)
```
in python code. f2py passes the array sizes,n and m, automatically to call fortran subroutines.
### check usage in python side ###
```
print mylib.vavbmc.__doc__
``` 
in python will give
```
c = vavbmc(a,b,[n,m])

Wrapper for ``vavbmc``.

Parameters
----------
a : input rank-1 array('d') with bounds (n)
b : input rank-1 array('d') with bounds (m)

Other Parameters
----------------
n : input int, optional
    Default: len(a)
m : input int, optional
    Default: len(b)

Returns
-------
c : rank-2 array('d') with bounds (n,m)
```

# Other f90-derived-class-support projects #
They are usually f2py wrappers. 

https://libatoms.github.io/QUIP/oo_fortran.html --- Object oriented interface on top of f2py generated wrappers.

https://pypi.python.org/pypi/f90wrap/0.1.4  --- Fortran to Python interface generator with derived type support
